In [1]:
from common.data_pipeline.dataset import DatasetChainer
from common.data_pipeline.MMCBNU_6000.dataset import DatasetLoader as mmcbnu
from common.data_pipeline.FV_USM.dataset import DatasetLoader as fvusm
from common.util.enums import EnvironmentType

datasets = DatasetChainer(datasets=[mmcbnu(included_portion=0.001), fvusm(included_portion=0)])
train, test, validation = datasets.get_dataset(EnvironmentType.PYTORCH)

2023-08-22 13:49:05,623 - root - INFO - Compiled train/test/validation split for MMCBNU6000
2023-08-22 13:49:05,666 - root - INFO - Compiled train/test/validation split for FV_USM
2023-08-22 13:49:05,667 - root - INFO - Concatenating train set
2023-08-22 13:49:05,668 - root - INFO - Concatenating test set
2023-08-22 13:49:05,669 - root - INFO - Concatenating validation set


In [2]:
from common.util.enums import DatasetSplitType


datasets.get_files(DatasetSplitType.TRAIN)

[DatasetObject(path='./datasets/MMCBNU_6000/ROIs/076/L_Fore/04.bmp', name='076/L_Fore/04.bmp', label=None, mask_path=None, metadata={'finger': 'fore', 'hand': 'left', 'is_augmented': False}),
 DatasetObject(path='./datasets/MMCBNU_6000/ROIs/030/R_Fore/05.bmp', name='030/R_Fore/05.bmp', label=None, mask_path=None, metadata={'finger': 'fore', 'hand': 'right', 'is_augmented': False}),
 DatasetObject(path='./datasets/MMCBNU_6000/ROIs/093/L_Fore/05.bmp', name='093/L_Fore/05.bmp', label=None, mask_path=None, metadata={'finger': 'fore', 'hand': 'left', 'is_augmented': False}),
 DatasetObject(path='./datasets/MMCBNU_6000/ROIs/006/L_Middle/08.bmp', name='006/L_Middle/08.bmp', label=None, mask_path=None, metadata={'finger': 'middle', 'hand': 'left', 'is_augmented': False})]